# Sydney Cytometry - Multicut segmentation with Ilastilk


#### Author
Dr. Thomas Myles Ashhurst (thomas.ashhurst@sydney.edu.au). Sydney Cytometry Facility, The Univeristy of Sydney and Centenary Institute.

#### Acknowledgements
This script and overall workflow are adapted from the script and workflow developed by Vito Zanotelli at the Bodenmiller laboratory (https://github.com/BodenmillerGroup/ImcSegmentationPipeline).

#### Description
This script is part of a modified pipeline for cell-segmentation of IMC data (based on the pipeline developed by the Bodenmiller lab). This script comprises 'Step 2' of the whole workflow. For instructions that cover the whole workflow, please visit https://wiki.centenary.org.au/x/kYBfCQ.

Please note, the full instruction set is provided on the Sydney Cytometry extranet site, which requires a log in to access. To gain access, please visit https://sydneycytometry.org.au/wiki-launch and hit the 'request access' button to get a login. Once you have a login, you can access the protocol page above.

#### Overview of the script
1. Follow the instructions in the 'user input' section to setup the script for your analysis. Modify the blocks as instructed, and then ***run*** the code block.
2. Once you have reached the end of the 'user input' section, ***run*** the subsequent code blocks to generate the output files. Run each block one by one, and keep an eye out for any instructions for checking outputs, or for any error messages that come up.

## User input ------------------------------------------------------------------------------------

### Import (i.e. load) necessary packages, including packages from 'IMC tools'

In [13]:
import os
import logging
import re
import zipfile

In [14]:
from imctools.scripts import ometiff2analysis
from imctools.scripts import imc2tiff
from imctools.scripts import ome2micat
from imctools.scripts import probablity2uncertainty
from imctools.scripts import convertfolder2imcfolder
from imctools.scripts import exportacquisitioncsv

### Configuration (needs to be adapted for use)
1. Change the text in ***red*** (right of the '=') to reflect the correct folder, file, or column names/patterns. Do not modify the black text (left of the '='). 
2. Once the edits have been made, run this code block to create variables that contain the name of various important folders, files, or column names.

**Before we begin, let's check out current working directory.**

Running the following code block should return the location of this script (which is your current working directory). By default this should be in the 'scripts' folder of your experiment folder.

For example, if my experiment folder is called ````TA123```` and it is located on my ````Desktop````, then the working directory might look like this (on a MacBook): ````/Users/thomasa/Desktop/TA123/scripts````.

In [15]:
os.getcwd()

'/Users/thomasa/OneDrive - The University of Sydney (Staff)/Library/Github (public)/SpectreMAP/workflows/Multicut segmentation/2_jupyter_script'

**Firstly, we must definte the location of the input data, the pattern of the file type that will be read in, and the location of the panel CSV file.**

By default this script is stored under the 'scripts' folder within your experiment folder. To tell python where to find other folders, '..' tells python to look in the parent directory, and '/foldername' tells it to look in another folder.

1. Input folders -- this is the location of the folder that ***contains*** the zipped folder of MCD files and text files. By default, this should be in the 'data' folder.

2. File regular expression -- this is the file type that python will look for in the data folder (specified above) -- the MCD files and corresponding ROI text files should be zipped into a single folder.

3. Folder base -- this instructs python to create a folder called 'output' where it will deposit the resulting files.

4. Panel -- here we need to tell the script where to find the 'panel.csv' file, which by default, should be contained in the 'config' folder. 

In [16]:
# 1) input folders
folders = ['../data']

# 2) file_regexp:
file_regexp = '.*.zip'

# 3) folder_base
folder_base = '../output'

# 4) panel
csv_pannel = '../1_panel/panel.csv'

Once you have edited and run the code block above, run each of the code blocks below to ensure the variables have been correctly assigned.

In [17]:
folders

['../data']

In [18]:
file_regexp

'.*.zip'

In [19]:
folder_base

'../output'

In [20]:
csv_pannel

'../1_panel/panel.csv'

**Secondly, we must define the columns contained within the panel CSV file.**

In [21]:
# Three columns are obligatory
    
    # metal columns
    csv_pannel_metal = 'Metal Tag' # Contains the isotope channel measured in the form (Metal)(Mass), e.g. Ir191, Yb171 etc.

    # ilastik columm: Bool, either 0 or 1: this selects channels to be used for cellular segmentation
    csv_pannel_ilastik = 'ilastik' 

    # full column: Contains the channels that should be quantified/measured in cellprofiler
    csv_pannel_full = 'full'

Once you have edited and run the code block above, run each of the code blocks below to ensure the variables have been correctly assigned.

In [22]:
csv_pannel_metal

'Metal Tag'

In [23]:
csv_pannel_ilastik

'ilastik'

In [24]:
csv_pannel_full

'full'

## End user input ------------------------------------------------------------------------------------
Do not modify any of the code blocks below. Rather, run each block one by one.

### Create other input variables (only change if really necessary and you know what your doing)

In [27]:
# parameters for resizing the images for ilastik
folder_analysis = os.path.join(folder_base, 'tiffs')
folder_ilastik = os.path.join(folder_base, 'ilastik')
folder_ome = os.path.join(folder_base, 'ometiff')
folder_cp = os.path.join(folder_base, 'cpout')
folder_histocat = os.path.join(folder_base, 'histocat')
folder_uncertainty = os.path.join(folder_base, 'uncertainty')

suffix_full = '_full'
suffix_ilastik = '_ilastik'
suffix_ilastik_scale = '_s2'
suffix_mask = '_mask.tiff'
suffix_probablities = '_Probabilities'


failed_images = list()

# Make a list of all the analysis stacks with format:
# (CSV_NAME, SUFFIX, ADDSUM)
# CSV_NAME: name of the column in the CSV to be used
# SUFFIX: suffix of the tiff
# ADDSUM: BOOL, should the sum of all channels be added as the first channel?
list_analysis_stacks =[
    (csv_pannel_ilastik, suffix_ilastik, 1),
    (csv_pannel_full, suffix_full, 0)
]

In [28]:
# Generate all the folders if necessary
for fol in [folder_base, folder_analysis, folder_ilastik,
            folder_ome, folder_cp, folder_histocat, folder_uncertainty]:
    if not os.path.exists(fol):
        os.makedirs(fol)

### Convert zipped IMC acquisitions to input format

This script works with zipped IMC acquisitions:
Each acquisition session = (1 mcd file) should be zipped in a folder containing:
- The `.mcd` file
- All associated `.txt` file generated during the acquisition of this `.mcd` file -> Don't change any of the filenames!!

In [29]:
# Convert mcd containing folders into imc zip folders

In [31]:
%%time
failed_images = list()
re_fn = re.compile(file_regexp)

for fol in folders:
    for fn in os.listdir(fol):
        if re_fn.match(fn):
            fn_full = os.path.join(fol, fn)
            print(fn_full)
            try:
                convertfolder2imcfolder.convert_folder2imcfolder(fn_full, out_folder=folder_ome,
                                                                   dozip=False)
            except:
                logging.exception('Error in {}'.format(fn_full))

../data/MCD file.zip
CPU times: user 22.3 s, sys: 16.6 s, total: 38.8 s
Wall time: 51.7 s


In [32]:
# Generate a csv with all the acquisition metadata
exportacquisitioncsv.export_acquisition_csv(folder_ome, fol_out=folder_cp)

In [33]:
# Convert ome.tiffs to a HistoCAT compatible format, e.g. to do some visualization and channel checking.

In [34]:
%%time
if not(os.path.exists(folder_histocat)):
    os.makedirs(folder_histocat)
for fol in os.listdir(folder_ome):
    ome2micat.omefolder2micatfolder(os.path.join(folder_ome,fol), folder_histocat, dtype='uint16')


CPU times: user 801 ms, sys: 1.34 s, total: 2.14 s
Wall time: 2.6 s


In [35]:
# Generate the analysis stacks

In [36]:
%%time
for fol in os.listdir(folder_ome):
    sub_fol = os.path.join(folder_ome, fol)
    for img in os.listdir(sub_fol):
        if not img.endswith('.ome.tiff'):
            continue
        basename = img.rstrip('.ome.tiff')
        print(img)
        for (col, suffix, addsum) in list_analysis_stacks:
            try:
                ometiff2analysis.ometiff_2_analysis(os.path.join(sub_fol, img), folder_analysis,
                                                basename + suffix, pannelcsv=csv_pannel, metalcolumn=csv_pannel_metal,
                                                usedcolumn=col, addsum=addsum, bigtiff=False,
                                               pixeltype='uint16')
            except:
                logging.exception('Error in {}'.format(img))
            



20171228_spleen315_500x500_editedforFAS_s1_p9_r4_a4_ac.ome.tiff
20171228_spleen315_500x500_editedforFAS_s1_p9_r18_a18_ac.ome.tiff
20171228_spleen315_500x500_editedforFAS_s1_p9_r2_a2_ac.ome.tiff
20171228_spleen315_500x500_editedforFAS_s1_p9_r8_a8_ac.ome.tiff
20171228_spleen315_500x500_editedforFAS_s1_p9_r28_a28_ac.ome.tiff
20171228_spleen315_500x500_editedforFAS_s1_p9_r6_a6_ac.ome.tiff
20171228_spleen315_500x500_editedforFAS_s1_p9_r30_a30_ac.ome.tiff
20171228_spleen315_500x500_editedforFAS_s1_p9_r32_a32_ac.ome.tiff
20171228_spleen315_500x500_editedforFAS_s1_p9_r26_a26_ac.ome.tiff
20171228_spleen315_500x500_editedforFAS_s1_p9_r12_a12_ac.ome.tiff
20171228_spleen315_500x500_editedforFAS_s1_p9_r10_a10_ac.ome.tiff
20171228_spleen315_500x500_editedforFAS_s1_p9_r24_a24_ac.ome.tiff
20171228_spleen315_500x500_editedforFAS_s1_p9_r14_a14_ac.ome.tiff
20171228_spleen315_500x500_editedforFAS_s1_p9_r20_a20_ac.ome.tiff
20171228_spleen315_500x500_editedforFAS_s1_p9_r22_a22_ac.ome.tiff
20171228_spleen315

#### Potential errors
If you see error messages here that relate to the metal or target names, then it is most likely there is a problem with the 'panel.csv' file. A key error can come about when one of the metal names is entered incorrectly. For example, having '141Pr' instead of 'Pr141'. Please check carefully, and see the troubleshooting section in the main workflow protocol (see link at the top of this script).


Bodenmiller protocol original notes:

An key-error here of the form:

```
line 91, in <listcomp> return [order_dict[m] for m in metallist] KeyError: 'Ru100'
```

Is an indication that the `panel.csv` contains a metal channel that was not actually measured in all or one of the acquisitions!
Please check the images as well as the `panel.csv` to make sure that only channels are in the `panel.csv` that were actually measured.

## Next steps ------------------------------------------------------------------------------------

Please visit https://wiki.centenary.org.au/x/kYBfCQ for the rest of the analysis workflow.